<a href="https://colab.research.google.com/github/comparativechrono/panel_app_gene_enrichment/blob/main/panel_app_gene_association.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="width:100%;">
  <tr>
    <td style="text-align:left;">
      <h1>PanelApp Gene Enrichment v1.7</h1>
      <p>Enrichment analysis using clinical gene panels</p>
    </td>
    <td style="text-align:right;">
      <img src="https://raw.githubusercontent.com/comparativechrono/panel_app_gene_enrichment/main/logo/logo.png" width="100" height="100" />
    </td>
  </tr>
</table>

In this notebook we do some simple enrichment on the Genomics England gene panels pulled from the PanelApp API. The code cycles through panel app panels, gets panel IDs, and obtains the gene symbols for the green genes on each panel. Then we provide some example gene sets for enrichment analysis, and look for whether any panels are enriched in these gene sets.

# Environment preparation

In [ ]:
import requests
import pandas as pd
from scipy.stats import fisher_exact
import matplotlib.pyplot as plt
from matplotlib_venn import venn3, venn3_circles
import numpy as np
import statsmodels.stats.multitest as multi
import math
import six
import seaborn as sns

# Step 1 - Data Acquisition
Get all the genes for each gene panel from the PanelApp API. Change confidence level to 2 for amber genes, currently set at 3 for green genes. This API call takes about 10 minutes to pull the whole database.

In [ ]:
def get_all_panels():
    # Define the base URL
    base_url = 'https://panelapp.genomicsengland.co.uk/api/v1/'

    # Define the endpoint path
    endpoint_path = 'panels/'

    # Create the full URL
    full_url = base_url + endpoint_path

    all_panels = []

    while full_url is not None:
        # Make the request
        response = requests.get(full_url)

        # Check that the request was successful
        if response.status_code == 200:
            # Parse the response as JSON
            panels = response.json()

            # Add the results to our list
            all_panels.extend(panels['results'])

            # Get the next URL
            full_url = panels['next']
        else:
            print(f'Request failed with status code {response.status_code}')
            return None

    return all_panels

def get_genes_for_panel(panel_id):
    # Define the base URL
    base_url = 'https://panelapp.genomicsengland.co.uk/api/v1/'

    # Define the endpoint path
    endpoint_path = f'panels/{panel_id}/genes/'

    # Create the full URL
    full_url = base_url + endpoint_path

    # List to store all genes for the panel
    all_genes = []

    while full_url is not None:
        # Make the request
        response = requests.get(full_url)

        # Check that the request was successful
        if response.status_code == 200:
            # Parse the response as JSON
            panel_data = response.json()

            # Add the genes to the list
            all_genes.extend([(gene['gene_data']['gene_symbol'], gene['confidence_level']) for gene in panel_data['results']])

            # Get the next URL
            full_url = panel_data['next']
        else:
            print(f'Request failed with status code {response.status_code}')
            return None

    return all_genes

# Get all panels
all_panels = get_all_panels()

if all_panels is not None:
    # Create a DataFrame from the list of panels
    df = pd.DataFrame(all_panels)

    # Create two empty dictionaries to store gene series
    genes_dict = {}
    all_genes_dict = {}

    # Iterate over the panels
    for panel_id in df['id']:
        # Initialize the list of green genes for this panel
        green_genes = []

        # Get the genes for the panel
        genes = get_genes_for_panel(panel_id)

        # Check if genes exist and is not an empty list
        if genes is not None and len(genes) > 0:
            # Split the tuples into two lists: genes and confidence levels
            gene_symbols, confidence_levels = zip(*genes)

            # Create a DataFrame for this panel
            panel_df = pd.DataFrame({
                'gene_symbol': gene_symbols,
                'confidence_level': confidence_levels
            })

            # Add the DataFrame to the all_genes_dict
            all_genes_dict[panel_id] = panel_df

            # Collect the green genes
            green_genes = [gene_symbol for gene_symbol, confidence_level in genes if int(float(confidence_level)) == 3]

        # Add the green genes to genes_dict regardless of whether there are any green genes or not
        genes_dict[panel_id] = pd.Series(list(set(green_genes)), dtype='object')

    # Concatenate all the DataFrames along the columns axis
    all_genes_df = pd.concat(all_genes_dict.values(), keys=all_genes_dict.keys(), axis=0)

    # Reset the index
    all_genes_df.reset_index(level=0, inplace=True)

    # Rename the 'level_0' column to 'panel_id'
    all_genes_df.rename(columns={'level_0': 'panel_id'}, inplace=True)

    # Save the all_genes_df DataFrame to a CSV file
    all_genes_df.to_csv('all_genes.csv')

    # Create a DataFrame from the dictionary of green genes
    genes_df = pd.concat(genes_dict, axis=1)

    # Save the genes_df DataFrame to a CSV file
    genes_df.to_csv('genes.csv')


# Step 2 - Gene Panel Enrichment Testing

Get panel names from original API request

In [ ]:
panel_id_to_name = {panel['id']: panel['name'] for panel in all_panels}


### Download your dataset for looking for enrichment
Three are used in the paper:

1.   circaDB rhythmic genes (http://circadb.hogeneschlab.org/human)
2.   maternal effect genes
3.   GO:0019318 - hexose metabolic process (https://amigo.geneontology.org/amigo/term/GO:0019318)

The gene lists are stored in the `examples` directory on GitHub


In [ ]:
# dataset_url = "https://raw.githubusercontent.com/comparativechrono/panel_app_gene_enrichment/main/examples/circaDB.tsv"

# dataset_url = "https://raw.githubusercontent.com/comparativechrono/panel_app_gene_enrichment/main/examples/maternal.tsv"

# dataset_url = "https://raw.githubusercontent.com/comparativechrono/panel_app_gene_enrichment/main/examples/0019318.tsv"

# dataset_url = "https://raw.githubusercontent.com/comparativechrono/panel_app_gene_enrichment/main/examples/seasonal_babydiet_dopico_S3.tsv"

dataset_url = "https://raw.githubusercontent.com/comparativechrono/panel_app_gene_enrichment/main/examples/GO0198738_human.tsv"

# Load gene set
gene_set = set(pd.read_csv(dataset_url, sep='\t', header=None)[0])

### Perform the enrichment using Fishers exact test

Two things are outputted from this code block. First, the results of Fisher's exact test, and the FDR corrected p value. Secondly, a file containing the genes in the overlap between the gene set being tested and the panels.

In [ ]:
# Calculate Fisher's test for each panel
results = []
overlapping_genes_dict = {}  # Create a dictionary to hold the overlapping genes
total_genes = 20449
for panel_id, panel_genes in genes_df.items():
    panel_genes = set(panel_genes.dropna())
    overlapping_genes = gene_set & panel_genes  # Get the overlapping genes
    in_gene_set_and_panel = len(gene_set & panel_genes)
    in_gene_set_and_not_panel = len(gene_set) - in_gene_set_and_panel
    in_not_gene_set_and_panel = len(panel_genes) - in_gene_set_and_panel
    in_not_gene_set_and_not_panel = total_genes - len(gene_set) - in_not_gene_set_and_panel

    contingency_table = [[in_gene_set_and_panel, in_gene_set_and_not_panel],
                         [in_not_gene_set_and_panel, in_not_gene_set_and_not_panel]]

    odds_ratio, p_value = fisher_exact(contingency_table)

    # Include panel name in results
    panel_name = panel_id_to_name[int(panel_id)]
    results.append((panel_id, panel_name, odds_ratio, p_value))

    # Store the overlapping genes for this panel
    overlapping_genes_dict[panel_name] = list(overlapping_genes)

# Create a DataFrame from the results
result_df = pd.DataFrame(results, columns=['Panel ID', 'Panel Name', 'Odds Ratio', 'p-value'])

# Perform FDR correction on the p-values
result_df['q-value'] = multi.multipletests(result_df['p-value'], method='fdr_bh')[1]

# Save the DataFrame to a CSV file
result_df.to_csv('fisher_results.csv')

# Convert the overlapping genes dictionary to a DataFrame
overlapping_genes_df = pd.DataFrame(dict([ (k,pd.Series(v, dtype=object)) for k,v in overlapping_genes_dict.items() ]))

# Save the DataFrame to a CSV file
overlapping_genes_df.to_csv('overlapping_genes.csv', index=False)



# Step 3 - Data Visualization

### Venn diagram
Create a Venn diagram of which genes in the gene set are green, amber or red on Genomics England PanelApp

In [ ]:
# Convert confidence level to integer
all_genes_df['confidence_level'] = all_genes_df['confidence_level'].astype(int)

# Get the sets of unique genes for each confidence level
green_genes = set(all_genes_df[all_genes_df['confidence_level'] == 3]['gene_symbol'].unique())
amber_genes = set(all_genes_df[all_genes_df['confidence_level'] == 2]['gene_symbol'].unique())
red_genes = set(all_genes_df[all_genes_df['confidence_level'] == 1]['gene_symbol'].unique())

# Only keep genes that are in gene_set
green_genes = green_genes.intersection(gene_set)
amber_genes = amber_genes.intersection(gene_set)
red_genes = red_genes.intersection(gene_set)

print(f"Number of green genes: {len(green_genes)}")
print(f"Number of amber genes: {len(amber_genes)}")
print(f"Number of red genes: {len(red_genes)}")

# Plot the Venn diagram
plt.figure(figsize=(10, 10))

# Use the venn3 function to create the Venn diagram and store the result
v = venn3([green_genes, amber_genes, red_genes], ('Green Genes', 'Amber Genes', 'Red Genes'))

# Change the colors
if len(green_genes) > 0:
    v.get_patch_by_id('100').set_color('green')
if len(amber_genes) > 0:
    v.get_patch_by_id('010').set_color('orange')
if len(red_genes) > 0:
    v.get_patch_by_id('001').set_color('red')

# Change the line styles
c = venn3_circles([green_genes, amber_genes, red_genes], linestyle='solid')
if len(green_genes) > 0:
    c[0].set_ls('solid')  # Change the style of the first circle line to solid
if len(amber_genes) > 0:
    c[1].set_ls('solid')  # Change the style of the second circle line to solid
if len(red_genes) > 0:
    c[2].set_ls('solid')  # Change the style of the third circle line to solid

# Save the plot to a file
plt.savefig("venn.png", dpi=300, bbox_inches='tight')

# Display the plot
plt.show()



## Volcano plot

Here we plot our output results using a simple volcano plot. The plots is scaled so that an OR of 1 is always in the centre of the plot. N.b. if a gene panel does not contain any of the genes on your list it will not be shown.

In [ ]:
def create_volcano_plot(df, column, significance_level=0.05, ylabel='', filename='volcano_plot.png'):
    # Transforming column into -log10(column)
    df['-log10(' + column + ')'] = -np.log10(df[column])

    # Creating boolean masks for significant and non-significant points
    sig = df[column] < significance_level
    not_sig = df[column] >= significance_level

    # Create the figure and axis objects
    fig, ax = plt.subplots()

    # Plot the significant points in blue
    sig_scatter = ax.scatter(df['Odds Ratio'][sig], df['-log10(' + column + ')'][sig], color='blue')

    # Plot the non-significant points in red
    not_sig_scatter = ax.scatter(df['Odds Ratio'][not_sig], df['-log10(' + column + ')'][not_sig], color='red')

    # Set the scale of x axis as log scale
    ax.set_xscale('log')

    # Filter out zeros, infinities, and NaN values before calculating max and min OR values
    filtered_OR = df['Odds Ratio'].replace([np.inf, -np.inf, 0], np.nan).dropna()

    # Take the reciprocal of any OR less than 1.
    reciprocal_OR = filtered_OR.copy()
    reciprocal_OR[filtered_OR < 1] = 1 / reciprocal_OR[filtered_OR < 1]

    # Look for the maximum x value between the OR values and the reciprocal OR values.
    max_OR = max(filtered_OR.max(), reciprocal_OR.max())

    # Round this value up to the nearest value on the log scale
    max_x = 10 ** (math.ceil(math.log10(max_OR)))

    # Use this value as max_x and take its reciprocal to be min_x
    min_x = 1 / max_x

    # Set the minimum and maximum value of x-axis
    ax.set_xlim([min_x, max_x])

    # Label the axes
    ax.set_xlabel('Odds Ratio')
    ax.set_ylabel(ylabel)

    # Add the legend
    ax.legend([sig_scatter, not_sig_scatter], ['Significant (' + column + '<0.05)', 'Not Significant'])

    # Save the plot to a file
    plt.savefig(filename, dpi=300, bbox_inches='tight')

    # Show the plot
    plt.show()

# Create the plots
create_volcano_plot(result_df, 'p-value', ylabel='-log10(p-value)', filename='volcano_plot_p.png')
create_volcano_plot(result_df, 'q-value', ylabel='-log10(q-value)', filename='volcano_plot_q.png')


### Summary of significant results
A styled table of top 10 most enriched panels

In [ ]:
# Filter the data to only include rows with q-value less than 0.05
filtered_df = result_df[result_df['q-value'] < 0.05]

# Sort the data by q-value in ascending order and take the top 10 rows
top_10_df = filtered_df.sort_values(by='q-value', ascending=True).head(10)

# Add a function to wrap text after 5 words
def wrap_text(text, line_length=5):
    words = text.split()
    return '\n'.join([' '.join(words[i:i+line_length]) for i in range(0, len(words), line_length)])

# Apply the function to the Panel Name column
top_10_df['Panel Name'] = top_10_df['Panel Name'].apply(wrap_text)

# Save the top 10 rows as a figure
def render_mpl_table(data, col_width=2.5, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors)])
        # Make Panel Name cell wider and wrap the text
        if k[1] == 1:
            cell.get_text().set_wrap(True)
            cell.set_width(5*col_width)
        else:  # This line sets the width for other cells
            cell.set_width(col_width)
    return fig, ax

# Convert 'Panel ID' to string
top_10_df['Panel ID'] = top_10_df['Panel ID'].astype(str)

def round_to_sigfigs(num, sig_figs):
    if np.isinf(num):
        return num  # if the value is np.inf or -np.inf, return it as is
    elif num != 0:
        return round(num, -int(math.floor(math.log10(abs(num))) - (sig_figs - 1)))
    else:
        return 0  # Can't take the log of 0

# Apply the function to the DataFrame
top_10_df = top_10_df.applymap(lambda x: round_to_sigfigs(x, 3) if isinstance(x, (int, float)) else x)

# Drop the '-log10(p-value)' and '-log10(q-value)' columns
top_10_df = top_10_df.drop(columns=['-log10(p-value)', '-log10(q-value)'])

fig, ax = render_mpl_table(top_10_df, header_columns=0, col_width=3)

# Save the figure
fig.savefig('top_10_significant_results.png')

# Display the figure in the notebook
plt.show()


### Make a clustergram
A heatmap of the genes in the top 10 most enriched panels

In [ ]:
# Get the panel names from the table
top_10_panels = top_10_df['Panel Name'].values

# Extract the columns of the top 10 panels from overlapping_genes_df
top_10_panels_df = overlapping_genes_df[top_10_panels]

# Concatenate the genes from all the top 10 panels into one Series, remove duplicates to get unique genes
unique_genes = pd.concat([top_10_panels_df[panel].dropna() for panel in top_10_panels]).unique()

# Initialize a new DataFrame with unique genes as index and top 10 panels as columns
binary_matrix = pd.DataFrame(index=unique_genes, columns=top_10_panels)

# Fill in the value for each gene as 1 if it is present in that panel, and 0 if not
for panel in top_10_panels:
    binary_matrix.loc[top_10_panels_df[panel].dropna().values, panel] = 1

# Fill the rest of the matrix with 0
binary_matrix.fillna(0, inplace=True)

# Plot a heatmap using the binary_matrix
plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
sns.heatmap(binary_matrix, cmap='Blues', cbar=False)
# plt.title("Gene Presence in Top 10 Panels")
plt.xlabel("Panel Name")
plt.ylabel("Gene")

# Adjust the layout
plt.tight_layout()

# Save the figure
plt.savefig('genes_top_10_significant_results.png')

# Show the figure
plt.show()
